<center> 
    <h1> Steganography Project </h1> 
    <h3> CS07552 - Cryptographic Algorithms </h3>
    <h3> Benny Liang </h3>
</center>

### Importing libraries

In [20]:
import os
import sys
import re
from typing import Union
from PIL import Image
import numpy as np
import hashlib
import base64
from random import seed, randint
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Hash import SHA512, SHA384, SHA256, SHA224
from cryptography.hazmat.primitives import padding
import cv2


print("System information: {0}".format(sys.version))

System information: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


### Input variables

In [21]:
BLOCK_SIZE = 128
MAX_NUM_BLOCKS_LENGTH = 16
ORIGINAL_IMAGE_PATH = "C:/Users/pawan/Desktop/Crytographic_algorithm/original.png"
MODIFIED_IMAGE_DIR = "C:/Users/pawan/Desktop/Crytographic_algorithm/"
PASSPHRASE = "CXSOToq3oY0r9wm1P0T1"


PASSPHRASE = PASSPHRASE.encode('utf-8')
IMAGE_NAME = re.search(r'(\.*?).png', ORIGINAL_IMAGE_PATH).group(1)
MODIFIED_IMAGE_PATH = "{0}{1}modified.png".format(MODIFIED_IMAGE_DIR, IMAGE_NAME)

In [22]:
### Viewing image details

In [23]:
image = Image.open(MODIFIED_IMAGE_PATH)
assert image.format == "PNG", "Please use a PNG instead of a {0:s}".format(image.format)
assert image.mode == "RGB" or image.mode == "RGBA", "Please use a RGB image instead of a {0:s}".format(image.mode)
print(image)

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=509x340 at 0x1C32C54D0C8>


### Generating the AES key and permutation key

In [24]:
def get_shaHash(data:Union[str, bytes], algorithm:int) -> hex:
    """
    Returns a hex representation of the hashed data using SHA224, SHA256, SHA384, or SHA512
    Arguments:
    data: \t the inputted data to be hashed. Expected types: str, and bytes. 
    algorithm: \t int representation of which SHA algorithm to use. Will defaults to SHA384.
    """
    shaHash = None
    try:
        if algorithm == 512:
            shaHash = SHA512.new()
        elif algorithm == 384:
            shaHash = SHA384.new()
        elif algorithm == 256:
            shaHash = SHA256.new()
        elif algorithm == 224:
            shaHash = SHA224.new()
    except:
        pass
        
    if shaHash == None:
        shaHash = SHA384.new()
    
    shaHash = SHA384.new()
    if isinstance(data, str):
        try:
            bytes(data).decode('utf-8')
        except (TypeError, UnicodeDecodeError) as exception:
            data = data.encode('utf-8')
    shaHash.update(data)
    return shaHash.hexdigest()

def bytes_toInt(data:bytes) -> int:
    if isinstance(data, bytes):
        return int.from_bytes(data, 'big')
    raise TypeError("key is of type {0}, but should be bytes".format(type(key)))

def int_toBytes(data:int):
    if isinstance(data, int):
        return data.to_bytes((data.bit_length() + 7) // 8, 'big')
    elif isinstance(data, bytes):
        return data
    raise TypeError("data is of type {0}, but should be int.".format(type(key)))

# Finds the SHA hash of the passphrase
sha_hash = get_shaHash(PASSPHRASE, 384)

# SHA hash is broken up into two parts, one is used for AES encryption while the other is used in permutation
# Both of these keys are stored as bytes
aes_key = int_toBytes(int(sha_hash[0:64], 16))
permutation_seed = int_toBytes(int(sha_hash[64:], 16))

assert int(sha_hash[0:64], 16) == bytes_toInt(aes_key), "bytes_toInt(key) does not work correctly."

In [25]:
c = sha_hash
print(c)
d = int(c, 16)
print(d)
e = d % (2**32 - 1)
print(e)
np.random.seed(e)

d80816bd0eb4eaa467aff42743551d2c79cae730ddb158a53b28bf110173f46ccd5f393e3b31479ee3e257ae2b187eea
33250305945766165614567972043915182779977822373536008717443449554706873132236606284842258673267338903514434791571178
1030118783


### Encrypting the message with AES

In [28]:
def aes_encrypt(data:Union[str, bytes], key:bytes) -> str:
    """
    Returns a string for an encrypted mesasge using the AES algorithm.
    The message is padded with PKCS7 to work with AES.
    Arguments:
    data: \t The data to be encrypted. Expected types of string or bytes.
    key: \t The key to be used with the AES algorithm. Expected type of bytes.
    """
    cipher = AES.new(key, AES.MODE_ECB)
    padder = padding.PKCS7(BLOCK_SIZE).padder()
    if isinstance(data, str):
        data = bytes(data)
    padded_data = padder.update(data)
    padded_data += padder.finalize()
    return cipher.encrypt(padded_data)


In [16]:
def aes_decrypt(encrypted_data:bytes, key:bytes) -> str:
    """
    Returns a string for the decrypted message using the AES algorithm.
    The padding on the message with PKCS7 is removed.
    Arguments:
    encrypted_data: \t The encrypted data using AES. Expected types of bytes.
    key: \t The key to be used with the AES algorithm. Expected type of bytes.
    """
    cipher = AES.new(key, AES.MODE_ECB)
    padded_data = cipher.decrypt(encrypted_data)
    unpadder = padding.PKCS7(BLOCK_SIZE).unpadder()
    data = unpadder.update(padded_data)
    data += unpadder.finalize()
    return data

assert aes_decrypt(aes_encrypt("MESSAGE".encode('utf-8'), aes_key), aes_key) == bytes("MESSAGE".encode('utf-8')), "Decrypt(encrypted_data) does not match the original data."

### Recovering Encrypted Data From Image

In [17]:
img = cv2.imread(MODIFIED_IMAGE_PATH)
num_blocksX = int(img.shape[1] / 8)
num_blocksY = int(img.shape[0] / 8)
total_blocks = num_blocksX * num_blocksY

blocks_used = np.zeros((num_blocksY, num_blocksX))

In [18]:
permutation_seed = int_toBytes(int(sha_hash[64:], 16))

# Establish header blocks

num_iterations = 1

# Using the permutation_seed select the first header block
permutation_value = int(get_shaHash(permutation_seed, 512), 16)
block_index = permutation_value % total_blocks
blockX = block_index % num_blocksX
blockY = int(block_index / num_blocksX)
blocks_used[blockY, blockX] = -1

print("Block index: {0}. Number iterations: {1}.".format(block_index, num_iterations))

# Using data from the first block, update the permutation_seed
pixelX_start = blockX * 8
pixelX_curr = pixelX_start

pixelY_start = blockY * 8
pixelY_curr = pixelY_start

update_seed_data = str(num_iterations)
for i in range(8):
    pixel = img[pixelY_curr + i][pixelX_curr + i]
    update_seed_data = update_seed_data + str(pixel[0] % 100) + str(pixel[1] % 100) + str(pixel[2] % 100)    
permutation_seed = int_toBytes(int(update_seed_data) ^ bytes_toInt(permutation_seed))

Block index: 1003. Number iterations: 1.


In [19]:
# Using the updated permutation seed, find the second header block
found_block = False
while not found_block:
    num_iterations += 1
    permutation_value = int(get_shaHash(permutation_seed, 512), 16)
    block_index = permutation_value % total_blocks
    blockX = block_index % num_blocksX
    blockY = int(block_index / num_blocksX)
    if blocks_used[blockY][blockX] == 0:
        found_block = True
        blocks_used[blockY][blockX] = -1
    else:
        permutation_seed = int_toBytes(bytes_toInt(permutation_seed) + num_iterations)
        
# Retrieves the number of data blocks there are from within pixels 17-31
pixelX_start = blockX * 8
pixelX_curr = pixelX_start
pixelY_start = blockY * 8
pixelY_curr = pixelY_start

update_seed_data = str(num_iterations)
for i in range(8):
    pixel = img[pixelY_curr][pixelX_curr + 1]
    update_seed_data = update_seed_data + str(pixel[0] % 100) + str(pixel[1] % 100) + str(pixel[2] % 100)
rgb_index = permutation_value % 3
pixelY_curr += 2

string = "0b"

for i in range(MAX_NUM_BLOCKS_LENGTH):
    pixel = img[pixelY_curr][pixelX_curr]
    string += str(int(pixel[rgb_index]) & 1)
    rgb_index = (rgb_index + 1) % 3
    pixelX_curr += 1
    if pixelX_curr % 8 == 0:
        pixelX_curr = pixelX_start
        pixelY_curr += 1
    
num_data_blocks = int(string, 2)

In [12]:
# Getting the encrypted data in the image
data = "0b"

for i in range(int(num_data_blocks * BLOCK_SIZE / 64)):
    # Finding a block to use
    found_block = False
    while not found_block:
        num_iterations += 1
        permutation_value = int(get_shaHash(permutation_seed, 512), 16)
        block_index = permutation_value % total_blocks
        blockX = block_index % num_blocksX
        blockY = int(block_index / num_blocksX)
        if blocks_used[blockY][blockX] == 0:
            found_block = True
            blocks_used[blockY][blockX] = 1
        else:
            permutation_seed = int_toBytes(bytes_toInt(permutation_seed) + num_iterations)

    rgb_index = permutation_value % 3
    pixelX_start = blockX * 8
    pixelX_curr = pixelX_start
    pixelY_start = blockY * 8
    pixelY_curr = pixelY_start
    
    update_seed_data = str(num_iterations)
    y = list(np.random.permutation(64))
    for i in range(64):
        a = int(y[i] / 8)
        b = int(y[i] % 8)
        pixelX_curr = a + pixelX_start
        pixelY_curr = b + pixelY_start
        pixel = img[pixelY_curr][pixelX_curr]
        data += str(int(pixel[rgb_index]) & 1)
        if pixelX_curr == pixelY_curr:
            update_seed_data = update_seed_data + str(pixel[0] % 100) + str(pixel[1] % 100) + str(pixel[2] % 100)
        rgb_index = (rgb_index + 1) % 3
        
        
    
    permutation_seed = int_toBytes(int(update_seed_data) ^ bytes_toInt(permutation_seed))
    

data_int = int(data, 2)
test = aes_decrypt(int_toBytes(data_int), aes_key)
print(test)

b"Two households, both alike in dignity, In fair Verona, where we lay our scene, From ancient grudge break to new mutiny, Where civil blood makes civil hands unclean. From forth the fatal loins of these two foes A pair of star-cross'd lovers take their life; Whose misadventured piteous overthrows Do with their death bury their parents' strife. The fearful passage of their death-mark'd love, And the continuance of their parents' rage, Which, but their children's end, nought could remove, Is now the two hours' traffic of our stage; The which if you with patient ears attend, What here shall miss, our toil shall strive to mend.Two households, both alike in dignity, In fair Verona, where we lay our scene, From ancient grudge break to new mutiny, Where civil blood makes civil hands unclean. From forth the fatal loins of these two foes A pair of star-cross'd lovers take their life; Whose misadventured piteous overthrows Do with their death bury their parents' strife. The fearful passage of th

In [13]:
y = list(np.random.permutation(64))
print(y)

[54, 48, 63, 0, 13, 36, 34, 28, 25, 31, 6, 9, 26, 20, 19, 7, 32, 59, 22, 61, 16, 11, 50, 52, 30, 56, 39, 53, 8, 45, 15, 1, 60, 33, 23, 46, 29, 62, 47, 2, 4, 5, 3, 49, 17, 38, 35, 55, 44, 12, 10, 51, 58, 24, 37, 14, 43, 42, 41, 57, 27, 18, 40, 21]
